In [ ]:
# Import libraries as required
import pandas as pd
# pip install scikit-learn
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import joblib
import numpy as np


In [2]:
df_accepted = pd.read_csv("C:\\Users\\teoni\\Downloads\\project_loan\\accepted_2007_to_2018Q4.csv")
print(df_accepted)

C:\Users\teoni\AppData\Local\Temp\ipykernel_24492\2777032942.py:1: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  df_accepted = pd.read_csv("C:\\Users\\teoni\\Downloads\\project_loan\\accepted_2007_to_2018Q4.csv")


                                                       id  member_id  \
0                                                68407277        NaN   
1                                                68355089        NaN   
2                                                68341763        NaN   
3                                                66310712        NaN   
4                                                68476807        NaN   
...                                                   ...        ...   
2260696                                          88985880        NaN   
2260697                                          88224441        NaN   
2260698                                          88215728        NaN   
2260699  Total amount funded in policy code 1: 1465324575        NaN   
2260700   Total amount funded in policy code 2: 521953170        NaN   

         loan_amnt  funded_amnt  funded_amnt_inv        term  int_rate  \
0           3600.0       3600.0           3600.0   36 months 

In [3]:
print(df_accepted.columns.tolist())

['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d', 'last_fico_range_high', 'last_fico_range_low', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'acc_now_delinq',

In [4]:
df_accepted_transformed = df_accepted[['id','loan_amnt','term','int_rate','installment','annual_inc','dti','delinq_2yrs','revol_util','total_acc','loan_status']]
print(df_accepted_transformed)

# loan_amt: loan amount
# term: loan term
# int_rate: interest rate: Higher rates imply higher risk customers
# installment: High installment relative to income can signal risk
# annual_income: lower income may correlate with default
# dti: debt-to-income ratio: higher means higher risk
# delinq_2yrs: past delinquencies in 2 years
# revol_util: credit utilization ratio where high usage of available credit may signal financial stress
# total_acc: total number of credit accounts where more accounts may idneicate creditworthiness (or overleveraging)
# loan_status: loan outcome (target variable which predicts whether the loan will default)

                                                       id  loan_amnt  \
0                                                68407277     3600.0   
1                                                68355089    24700.0   
2                                                68341763    20000.0   
3                                                66310712    35000.0   
4                                                68476807    10400.0   
...                                                   ...        ...   
2260696                                          88985880    40000.0   
2260697                                          88224441    24000.0   
2260698                                          88215728    14000.0   
2260699  Total amount funded in policy code 1: 1465324575        NaN   
2260700   Total amount funded in policy code 2: 521953170        NaN   

               term  int_rate  installment  annual_inc    dti  delinq_2yrs  \
0         36 months     13.99       123.03     55000.0   

In [5]:
# Cleaning - since last 2 rows are aggregates and summations of total values, we drop the last 2 rows
df_accepted_transformed.drop(df_accepted_transformed.tail(2).index,inplace=True)
print(df_accepted_transformed)


               id  loan_amnt        term  int_rate  installment  annual_inc  \
0        68407277     3600.0   36 months     13.99       123.03     55000.0   
1        68355089    24700.0   36 months     11.99       820.28     65000.0   
2        68341763    20000.0   60 months     10.78       432.66     63000.0   
3        66310712    35000.0   60 months     14.85       829.90    110000.0   
4        68476807    10400.0   60 months     22.45       289.91    104433.0   
...           ...        ...         ...       ...          ...         ...   
2260694  89885898    24000.0   60 months     12.79       543.50     95000.0   
2260695  88977788    24000.0   60 months     10.49       515.74    108000.0   
2260696  88985880    40000.0   60 months     10.49       859.56    227000.0   
2260697  88224441    24000.0   60 months     14.49       564.56    110000.0   
2260698  88215728    14000.0   60 months     14.49       329.33     95000.0   

           dti  delinq_2yrs  revol_util  total_acc 

C:\Users\teoni\AppData\Local\Temp\ipykernel_24492\2770959849.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_accepted_transformed.drop(df_accepted_transformed.tail(2).index,inplace=True)


In [6]:
# Cleaning - drop null values (if any):
df_accepted_transformed.isnull().values.any()

True

In [8]:
# Cleaning - drop null values (if any) - dropped 10,000 values (which is 0.5% of values of data, hence shouldn't be a factor)
df_accepted_transformed = df_accepted_transformed.dropna() 
df_accepted_transformed.describe()
print(df_accepted_transformed)

               id  loan_amnt        term  int_rate  installment  annual_inc  \
0        68407277     3600.0   36 months     13.99       123.03     55000.0   
1        68355089    24700.0   36 months     11.99       820.28     65000.0   
2        68341763    20000.0   60 months     10.78       432.66     63000.0   
3        66310712    35000.0   60 months     14.85       829.90    110000.0   
4        68476807    10400.0   60 months     22.45       289.91    104433.0   
...           ...        ...         ...       ...          ...         ...   
2260694  89885898    24000.0   60 months     12.79       543.50     95000.0   
2260695  88977788    24000.0   60 months     10.49       515.74    108000.0   
2260696  88985880    40000.0   60 months     10.49       859.56    227000.0   
2260697  88224441    24000.0   60 months     14.49       564.56    110000.0   
2260698  88215728    14000.0   60 months     14.49       329.33     95000.0   

           dti  delinq_2yrs  revol_util  total_acc 

In [9]:
# Feature engineering:
# (1) Loan Status --> Y variable is loan_status, we need a variable (Fully Paid and Current as 0, where Charged Off as 1 - defaulted on loan)
df_accepted_transformed['loan_status'] = df_accepted_transformed['loan_status'].map(lambda x: 1 if x == 'Charged Off' else 0)
df_accepted_transformed['term'] = df_accepted_transformed['term'].str.replace(" months", "")
print(df_accepted_transformed)


               id  loan_amnt term  int_rate  installment  annual_inc    dti  \
0        68407277     3600.0   36     13.99       123.03     55000.0   5.91   
1        68355089    24700.0   36     11.99       820.28     65000.0  16.06   
2        68341763    20000.0   60     10.78       432.66     63000.0  10.78   
3        66310712    35000.0   60     14.85       829.90    110000.0  17.06   
4        68476807    10400.0   60     22.45       289.91    104433.0  25.37   
...           ...        ...  ...       ...          ...         ...    ...   
2260694  89885898    24000.0   60     12.79       543.50     95000.0  19.61   
2260695  88977788    24000.0   60     10.49       515.74    108000.0  34.94   
2260696  88985880    40000.0   60     10.49       859.56    227000.0  12.75   
2260697  88224441    24000.0   60     14.49       564.56    110000.0  18.30   
2260698  88215728    14000.0   60     14.49       329.33     95000.0  23.36   

         delinq_2yrs  revol_util  total_acc  loan_s

In [11]:
X = df_accepted_transformed.drop(columns=['loan_status', 'id'])
y = df_accepted_transformed['loan_status']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [12]:
# 1. Try LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

# Measures of model accuracy / classification report accuracy:
print(f"Model Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:\n", classification_report(y_test,y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test,y_pred))

# This generated all results with 0 indicators (of 0 with no defaults). We now try using other models such as Random Forest

c:\Users\teoni\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Model Accuracy: 88.05%
Classification Report:
               precision    recall  f1-score   support

           0       0.88      1.00      0.94    397554
           1       0.23      0.00      0.00     53878

    accuracy                           0.88    451432
   macro avg       0.55      0.50      0.47    451432
weighted avg       0.80      0.88      0.82    451432

Confusion Matrix:
 [[397437    117]
 [ 53844     34]]


In [ ]:
# 2. Try Decision Trees - good for quick predictions, but struggles with overfitting.


model2 = DecisionTreeClassifier(max_depth=5, min_samples_split=20, class_weight="balanced", random_state=42)
model2.fit(X_train,y_train)
y_pred = model2.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.5227741941200447
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.49      0.64    397554
           1       0.17      0.80      0.28     53878

    accuracy                           0.52    451432
   macro avg       0.56      0.64      0.46    451432
weighted avg       0.85      0.52      0.60    451432

Confusion Matrix:
 [[193076 204478]
 [ 10957  42921]]


In [14]:
# 3. Try RandomForest as it can handle imbalanced classes better
model3 = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state = 42)
model3.fit(X_train, y_train)
y_pred = model3.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

# Measures of model accuracy / classification report accuracy:
print(f"Model Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:\n", classification_report(y_test,y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test,y_pred))

# Now we see that Model Accuracy = 88.03% with a Random Forest model

Model Accuracy: 88.02%
Classification Report:
               precision    recall  f1-score   support

           0       0.88      1.00      0.94    397554
           1       0.38      0.01      0.01     53878

    accuracy                           0.88    451432
   macro avg       0.63      0.50      0.47    451432
weighted avg       0.82      0.88      0.83    451432

Confusion Matrix:
 [[397068    486]
 [ 53575    303]]


In [15]:
# Get probability predictions instead of direct class predictions
y_probs = model3.predict_proba(X_test)[:, 1]  # Probability of Default (Class 1)

# Adjust threshold (default is 0.5)
threshold = 0.3  # Try 0.3, 0.25, or 0.2
y_pred_adjusted = (y_probs >= threshold).astype(int)

# Recalculate classification metrics
print("Updated Classification Report:\n", classification_report(y_test, y_pred_adjusted))
print("Updated Confusion Matrix:\n", confusion_matrix(y_test, y_pred_adjusted))


Updated Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.96      0.92    397554
           1       0.30      0.14      0.19     53878

    accuracy                           0.86    451432
   macro avg       0.60      0.55      0.56    451432
weighted avg       0.82      0.86      0.84    451432

Updated Confusion Matrix:
 [[380353  17201]
 [ 46476   7402]]


In [ ]:
# accuracy - but this may not be the best model as this is an imbalanced classification model (where loan default is much rarer than full repayment)
# prediction - how many of the predicted defaults are actually defaults?
# recall - how many actual defaults did the model correctly catch?
# F1 score - balance between precision & recall (ueeful for imbalanced data)

In [17]:
# Use the model built for model predictions

joblib.dump(model3, 'loan_default_model.pkl')
model3 = joblib.load('loan_default_model.pkl')
predictions = model3.predict(X_test)

In [18]:
# Test the model against a sample input data 

new_application = np.array([15000, 36, 12.5, 400, 70000, 25, 0, 50, 10]).reshape(1, -1)
feature_names = ['loan_amnt', 'term', 'int_rate', 'installment', 'annual_inc', 
                 'dti', 'delinq_2yrs', 'revol_util', 'total_acc']
new_application_df = pd.DataFrame(new_application, columns=feature_names)

# Output result - whether will default / not default & probability of default

prediction = model3.predict(new_application_df)
print("Prediction:", "Default" if prediction[0] == 1 else "No Default")
probabilities = model3.predict_proba(new_application_df)
print(f"Probability of Default: {probabilities[0][1]:.2%}")

Prediction: No Default
Probability of Default: 19.00%
